In [1]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

#from samples.cats_dogs import cats_dogs
from samples.miyukiCamera import miyukiCamera

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")



Using TensorFlow backend.


In [7]:
config = miyukiCamera.MiyukiCameraConfig()
DATA_DIR = "/Users/donchan/Documents/Miyuki/MaskRCNN_data/datasets/mySelect"  #os.path.join(ROOT_DIR, "datasets/cats_dogs")

In [3]:
# Override the training configurations with a few
# changes for inferencing.
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                18
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [4]:
# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/cpu:0"  # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# TODO: code for 'training' test mode not ready yet
TEST_MODE = "inference"

In [5]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [8]:
# Load validation dataset
dataset = miyukiCamera.MiyukiCameraDataset()
dataset.load_classObjects(DATA_DIR, "val")

# Must call before using the dataset
dataset.prepare()

print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

/Users/donchan/Documents/Miyuki/MaskRCNN_data/datasets/mySelect/val
Images: 11
Classes: ['BG', 'drug', 'prescription', 'hands', 'sheets', 'money', 'cointab', 'hair']


In [12]:
# Create model in inference mode
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config)

In [13]:
print(model)

In [11]:


# Or, load the last model you trained
weights_path = model.find_last()[1]

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

FileNotFoundError: [Errno 2] Could not find model directory under /Users/donchan/Documents/Miyuki/Mask_RCNN/logs

In [16]:
image_id = random.choice(dataset.image_ids)
image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)
info = dataset.image_info[image_id]
print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id, 
                                       dataset.image_reference(image_id)))

print(len( [image]  ))

# Run object detection
results = model.detect([image], verbose=1)

# Display results
ax = get_ax(1)
r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            dataset.class_names, r['scores'], ax=ax,
                            title="Predictions")
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

## image info source  miyukiCamera
# index # 0 polygon
{'name': 'polygon', 'all_points_x': [52, 15, 109, 262, 491, 535, 566, 545, 481, 488, 52], 'all_points_y': [132, 580, 669, 702, 713, 404, 182, 122, 108, 162, 132]}
# index # 1 polygon
{'name': 'polygon', 'all_points_x': [625, 603, 595, 581, 562, 544, 531, 538, 547, 525, 523, 549, 585, 615, 612, 651, 674, 689, 735, 690, 701, 768, 885, 917, 941, 959, 981, 960, 961, 952, 625], 'all_points_y': [38, 105, 134, 176, 209, 240, 311, 344, 345, 385, 410, 437, 435, 386, 445, 468, 464, 437, 356, 447, 480, 481, 287, 310, 300, 253, 218, 114, 35, 33, 38]}
info object of image_info :  {'classification': 'drug'}
info object of image_info :  {'classification': 'hands'}
image ID: miyukiCamera.prescription-20180922-123117.jpg (7) /Users/donchan/Documents/Miyuki/MaskRCNN_data/datasets/mySelect/val/prescription-20180922-123117.jpg
1


AssertionError: len(images) must be equal to BATCH_SIZE